<a href="https://colab.research.google.com/github/Keoni808/NFL_Data_Cleaning/blob/main/NFL_Plays_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and Imports

In [ ]:
# Mount your Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Grab data from database
from google.cloud import bigquery

# Loading Data


In [ ]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## All Plays 2023

In [ ]:
nfl_2023_plays_query = """
                       SELECT *
                       FROM `nfl-data-430702.NFL_Scores.NFL-Plays-2023`
                       """

# Run the query, and return a pandas DataFrame
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_2023_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_2023_plays_query, job_config=safe_config)

# df_nfl_scores_data = safe_config_query.to_dataframe()

This query will process 11827487 bytes.


In [ ]:
df_2023_plays = safe_config_query.to_dataframe()

In [ ]:
df_2023_plays.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,1,0,Kickoff,E.McPherson kicks 67 yards from CIN 35 to GB -...,Kickoff from CIN 35
1,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,2,0,6 Yard Pass,(14:56) (Shotgun) J.Love pass short right to A...,1st & 10 at GB 18
2,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,3,0,8 Yard Pass,(14:20) J.Love pass short left to L.Musgrave p...,2nd & 4 at GB 24
3,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,4,0,3 Yard Run,(13:43) A.Dillon left guard to GB 35 for 3 yar...,1st & 10 at GB 32
4,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,5,0,Pass Incomplete,(13:06) (Shotgun) J.Love pass incomplete deep ...,2nd & 7 at GB 35


##Super Bowl Plays 2023

In [ ]:
nfl_2023_sb_plays_query = """
                          SELECT *
                          FROM `nfl-data-430702.NFL_Scores.NFL-Plays-SuperBowl-2023`
                          """

# Run the query, and return a pandas DataFrame
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_2023_sb_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_2023_sb_plays_query, job_config=safe_config)

# df_nfl_scores_data = safe_config_query.to_dataframe()

This query will process 41291 bytes.


In [ ]:
df_2023_plays_sb = safe_config_query.to_dataframe()

In [ ]:
df_2023_plays_sb.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,1,0,-3 Yard Run,(12:15) (Shotgun) I.Pacheco left guard to KC 2...,1st & 10 at KC 27
1,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,2,0,1 Yard Pass,(11:39) (Shotgun) P.Mahomes pass short left to...,2nd & 13 at KC 24
2,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,3,0,8 Yard Pass,(11:04) (Shotgun) P.Mahomes pass short right t...,3rd & 12 at KC 25
3,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,4,0,Punt,"(10:24) T.Townsend punts 43 yards to SF 24, Ce...",4th & 4 at KC 33
4,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,1,0,10 Yard Run,(6:28) (Shotgun) I.Pacheco right guard to KC 2...,1st & 10 at KC 11


# Isolating Different Plays
- The goal here is to parse out the different values for 'PlayOutcome'
  - separate pass / run / kickoff / etc.

## Parsing Plays


In [ ]:
df_2023_pass_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Pass')]
df_2023_run_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Run')]
df_2023_punt_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Punt')]
df_2023_sack_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Sack')]
df_2023_kickoff_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Kickoff')]
df_2023_fumble_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Fumble')]
df_2023_interception_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Interception')]
df_2023_penalty_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Penalty')]
df_2023_fieldgoal_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Field Goal')]
df_2023_touchdown_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Touchdown')]
df_2023_extrapoint_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Extra Point')]

plays_list = [df_2023_pass_sb,
              df_2023_run_sb,
              df_2023_punt_sb,
              df_2023_sack_sb,
              df_2023_kickoff_sb,
              df_2023_fumble_sb,
              df_2023_interception_sb,
              df_2023_penalty_sb,
              df_2023_fieldgoal_sb,
              df_2023_touchdown_sb,
              df_2023_extrapoint_sb]

## Sanity Check (All Plays Accounted for)

In [ ]:
df_check = pd.DataFrame()
for i in plays_list:
  df_check = pd.concat([df_check, i])

In [ ]:
df_check = df_check.sort_index()

In [ ]:
df_2023_plays_sb.equals(df_check)

True

# PlayDescription Feature Breakdown

ISSUES:
- laterals?
- penalties
  - Declined or accepted?
  - What kind of penalty was it?

- I need a check to make sure that all plays have been broken down
  - Possibly add a check for each type of play that happened?



## Pass Plays

In [ ]:
df_2023_pass_sb['PlayOutcome'].unique()

array(['1 Yard Pass', '8 Yard Pass', 'Pass for No Gain', '7 Yard Pass',
       '52 Yard Pass', 'Pass Incomplete', '10 Yard Pass', '9 Yard Pass',
       '5 Yard Pass', '18 Yard Pass', '11 Yard Pass', '12 Yard Pass',
       '-8 Yard Pass', '6 Yard Pass', '21 Yard Pass', '3 Yard Pass',
       '2 Yard Pass', '16 Yard Pass', '13 Yard Pass', '25 Yard Pass',
       '22 Yard Pass', '-3 Yard Pass', '4 Yard Pass', '19 Yard Pass',
       '17 Yard Pass', '20 Yard Pass', '23 Yard Pass', '24 Yard Pass'],
      dtype=object)

In [ ]:
# 3 different formats as far as I can see.
# 1. '# Yard Pass'
# 2. 'Pass Incomplete'
# 3. 'Pass for No Gain'

df_successful_passes = df_2023_pass_sb[df_2023_pass_sb['PlayOutcome'].str.contains('Yard Pass')]
df_incomplete_passes = df_2023_pass_sb[df_2023_pass_sb['PlayOutcome'].str.contains('Pass Incomplete')]
df_pass_for_no_gain = df_2023_pass_sb[df_2023_pass_sb['PlayOutcome'].str.contains('Pass for No Gain')]

In [ ]:
df_successful_passes.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
1,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,2,0,1 Yard Pass,(11:39) (Shotgun) P.Mahomes pass short left to...,2nd & 13 at KC 24
2,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,3,0,8 Yard Pass,(11:04) (Shotgun) P.Mahomes pass short right t...,3rd & 12 at KC 25
10,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,2,0,7 Yard Pass,(14:48) P.Mahomes pass short left to I.Pacheco...,1st & 10 at KC 25
13,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,5,0,52 Yard Pass,(13:01) (Shotgun) P.Mahomes pass deep right to...,1st & 10 at KC 39
21,2023,Super Bowl,SUN,02/11,49ers,Chiefs,3RD QUARTER,1,KC,0,3,0,10 Yard Pass,(14:15) (Shotgun) P.Mahomes pass short middle ...,2nd & 22 at KC 13


In [ ]:
# I am trying to figure out a way to make sure that all plays within 'df_successful_passes' have been accounted for.
#   1. Maybe I should create a new dataframe with all of the broken down data and compare the length to 'df_successful_passes'?
#       - The only issue that I see here is that I would need to find out some way to make sure that the indexing
#         is the same for both dataframes.
#         CHECKS:
#           1. (len(df_successful_passes) == len(df_detailed_successful_passes))
#           2. index of df_successful_passes == index of df_detailed_succesfull_passes
#               - I would need to make sure that 'df_detailed_successful_passes' has the same order as df_successful_passes

#   2. I could add all columns to a copy of df_successful_passes and add each broken down data as it is happening.
#     - I am unsure that all rows will be accounted for.

In [ ]:
new_columns_passing = ["TimeOnTheClock", "Formation", "Quarterback", "Receiver", "PassType", "PassDirection", "PassYardage",
                       "Tackler1", "Tackler2", "PressureBy",
                       "InjuriesOnPlay", "InjuredPlayers",
                       "PenaltyOnPlay", "PenaltyPlayers"]

passing_string_columns = ["TimeOnTheClock", "Formation", "Quarterback", "Receiver", "PassType", "PassDirection",
                          "Tackler1", "Tackler2", "PressureBy", "InjuredPlayers", "PenaltyPlayers"]

passing_int_columns = ["PassYardage", "InjuriesOnPlay", "PenaltyOnPlay"]

df_successful_passes_detailed = df_successful_passes.copy()
df_successful_passes_detailed = df_successful_passes_detailed.reindex(columns=df_successful_passes_detailed.columns.tolist() + new_columns_passing)
df_successful_passes_detailed[passing_string_columns] = df_successful_passes_detailed[passing_string_columns].astype(str)
df_successful_passes_detailed[passing_int_columns] = df_successful_passes_detailed[passing_int_columns].astype(float)
df_successful_passes_detailed["InjuredPlayers"] = df_successful_passes_detailed["InjuredPlayers"].astype(object)

In [ ]:
#######################
# REGULAR EXPRESSIONS #
#######################

time_on_clock_pattern = r'\(\d*:\d+\)'
formation = r'\([A-Za-z]+\)'
name_pattern = r'\b[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\b'
defense_tackler_1_name_pattern = r'\([A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*' # Will have a "(" in front of the name
defense_tackler_2_name_pattern = r' [A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\)' # Will have a ")" at the end of the name
defense_pressure_name_pattern = r'\[[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\]' # Surrounded by "[]" brackets
injured_or_penalty_name = r'[A-Za-z]+-[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*'

In [ ]:
for idx, value in df_successful_passes['PlayOutcome'].items():
  play = df_2023_plays_sb['PlayDescription'].iloc[idx]

  ################
  # Play details #
  ################

  # TimeOnTheClock
  TimeOnTheClock = re.findall(time_on_clock_pattern, play)
  df_successful_passes_detailed.loc[idx, 'TimeOnTheClock'] = TimeOnTheClock[0][1:-1]

  #############
  #  OFFENSE  #
  #############

  # Formation
  Formation = re.findall(formation, play)
  if len(Formation) > 0:
    df_successful_passes_detailed.loc[idx, 'Formation'] = Formation[0][1:-1]
  # Quarterback & Receiver
  Quarterback_Receiver = re.findall(name_pattern, play)
  df_successful_passes_detailed.loc[idx, 'Quarterback'] = Quarterback_Receiver[0] # Quarterback
  df_successful_passes_detailed.loc[idx, 'Receiver'] = Quarterback_Receiver[1] # Receiver
  # Deep or short pass
  if int(value.split()[0]) >= 20:
    df_successful_passes_detailed.loc[idx, 'PassType'] = 'Deep'
  else:
    df_successful_passes_detailed.loc[idx, 'PassType'] = 'Short'
  # Pass Direction
  if play.find('left') != -1:
    df_successful_passes_detailed.loc[idx, 'PassDirection'] = 'Left'
  elif play.find('right') != -1:
    df_successful_passes_detailed.loc[idx, 'PassDirection'] = 'Right'
  elif play.find('middle') != -1:
    df_successful_passes_detailed.loc[idx, 'PassDirection'] = 'Middle'
  # Yardage gained on play
  df_successful_passes_detailed.loc[idx, 'PassYardage'] = int(value.split()[0])

  #############
  #  DEFENSE  #
  #############

  tackler_1 = re.findall(defense_tackler_1_name_pattern, play) # tackler #1 (Could be solo or the one who initiated the hit)
  if len(tackler_1) > 0:
    df_successful_passes_detailed.loc[idx, 'Tackler1'] = tackler_1[0][1:]
  tackler_2 = re.findall(defense_tackler_2_name_pattern, play) # tackler #2 (equally contributed or assisted with tackle)
  if len(tackler_2) > 0:
    df_successful_passes_detailed.loc[idx, 'Tackler2'] = tackler_2[0][1:-1]
  pressure = re.findall(defense_pressure_name_pattern, play)   # Player who applied pressure to passer
  if len(pressure) > 0:
    df_successful_passes_detailed.loc[idx, 'PressureBy'] = pressure[0][1:-1]

  #############
  #  PENALTY  #
  #############

  if play.find('Penalty') != -1:
    df_successful_passes_detailed.loc[idx, 'PenaltyOnPlay'] = 1
    penalty_name = re.findall(injured_or_penalty_name, play)
    df_successful_passes_detailed.at[idx, 'PenaltyPlayers'] = penalty_name
    # for i in penalty_name:
    #   df_successful_passes_detailed.loc[idx, 'PenaltyPlayers'] = i

  if play.find('injured') != -1:
    df_successful_passes_detailed.loc[idx, 'InjuriesOnPlay'] = 1
    injured_name = re.findall(injured_or_penalty_name, play)
    df_successful_passes_detailed.at[idx, 'InjuredPlayers'] = injured_name

In [ ]:
df_successful_passes_detailed[["PlayDescription", "Quarterback", "Receiver", "PassType", "PassDirection", "PassYardage", "Tackler1", "Tackler2", "PressureBy", "InjuriesOnPlay", "InjuredPlayers", "PenaltyOnPlay", "PenaltyPlayers"]]

,PlayDescription,Quarterback,Receiver,PassType,PassDirection,PassYardage,Tackler1,Tackler2,PressureBy,InjuriesOnPlay,InjuredPlayers,PenaltyOnPlay,PenaltyPlayers
1,(11:39) (Shotgun) P.Mahomes pass short left to...,P.Mahomes,T.Kelce,Short,Left,1.0,C.Young,D.Greenlaw,nan,NaN,nan,NaN,nan
2,(11:04) (Shotgun) P.Mahomes pass short right t...,P.Mahomes,J.McKinnon,Short,Right,8.0,F.Warner,D.Greenlaw,nan,NaN,nan,NaN,nan
10,(14:48) P.Mahomes pass short left to I.Pacheco...,P.Mahomes,I.Pacheco,Short,Left,7.0,T.Gipson,nan,nan,NaN,nan,NaN,nan
13,(13:01) (Shotgun) P.Mahomes pass deep right to...,P.Mahomes,M.Hardman,Deep,Right,52.0,J.Brown,nan,nan,NaN,nan,NaN,nan
21,(14:15) (Shotgun) P.Mahomes pass short middle ...,P.Mahomes,N.Gray,Short,Middle,10.0,L.Ryan,nan,nan,NaN,nan,NaN,nan
24,(12:26) (Shotgun) P.Mahomes pass short right t...,P.Mahomes,T.Kelce,Short,Right,9.0,O.Burks,C.Ward,nan,NaN,nan,NaN,nan
28,(3:26) (Shotgun) P.Mahomes pass short middle t...,P.Mahomes,T.Kelce,Short,Middle,5.0,O.Burks,nan,nan,NaN,nan,NaN,nan
33,(14:24) B.Purdy pass short right to K.Juszczyk...,B.Purdy,K.Juszczyk,Short,Right,18.0,N.Bolton,nan,nan,NaN,nan,NaN,nan
34,(13:41) (Shotgun) B.Purdy pass short middle to...,B.Purdy,C.McCaffrey,Short,Middle,11.0,L.Chenal,nan,nan,NaN,nan,NaN,nan
37,(10:16) (Shotgun) B.Purdy pass short middle to...,B.Purdy,B.Aiyuk,Short,Middle,18.0,J.Reid,nan,nan,NaN,nan,NaN,nan


In [ ]:
df_successful_passes_detailed["Quarterback"].unique()

array(['P.Mahomes', 'B.Purdy'], dtype=object)

In [ ]:
df_successful_passes_detailed[df_successful_passes_detailed["Quarterback"] == "B.Purdy"]["PassYardage"].sum()

245.0

In [ ]:
df_successful_passes_detailed[df_successful_passes_detailed["Quarterback"] == "B.Purdy"]["Receiver"].unique()

array(['K.Juszczyk', 'C.McCaffrey', 'B.Aiyuk', 'D.Samuel', 'J.Jennings',
       'C.Conley', 'R.McCloud', 'G.Kittle'], dtype=object)

In [ ]:
df_successful_passes_detailed[df_successful_passes_detailed["Receiver"] == "G.Kittle"]["PassYardage"].sum()

4.0